# Generate UUID report on all public primary protected datasets
This notebook builds a table that has two important fields

* `has_uuids` - False if there are no UUIDs present in the UUID-API for this dataset
* `number_of_uuids` - the number of UUIDs in the UUID-API db.

In [ ]:
import hubmapbags
from datetime import datetime
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from warnings import warn as warning

token = ''
instance = 'prod' #default instance is test

In [ ]:
# get assay types
assay_names = hubmapbags.get_assay_types()
print(assay_names)
assay_names = ['ATACseq-bulk']

In [ ]:
report = pd.DataFrame()
for assay_name in assay_names:
    print(assay_name)
    datasets = pd.DataFrame(hubmapbags.get_hubmap_ids( assay_name=assay_name, token=token ))

    if datasets.empty:
        continue
    
    #clean up
    datasets = datasets[(datasets['data_type'] != 'image_pyramid')]
    datasets = datasets[(datasets['status'] == 'Published')]
    datasets = datasets[(datasets['is_protected'] == True)]

    datasets['has_uuids'] = None
    datasets['number_of_uuids'] = None
    datasets['directory'] = None
    datasets['number_of_files'] = None
    for index, datum in tqdm(datasets.iterrows()):
        
        if hubmapbags.apis.is_protected( datum['hubmap_id'], token=token ):
            datasets.loc[index, 'number_of_uuids'] = hubmapbags.uuids.get_number_of_uuids( datum['hubmap_id'], token=token )

            if datasets.loc[index, 'number_of_uuids'] == 0:
                datasets.loc[index, 'has_uuids'] = False
            else:
                datasets.loc[index, 'has_uuids'] = True
                
            datasets.loc[index, 'directory'] = hubmapbags.apis.get_directory( datum['hubmap_id'], instance='prod', token=token )
            datasets.loc[index, 'number_of_files'] = hubmapbags.apis.get_number_of_files( datum['hubmap_id'], instance='prod', token=token )
    
    if report.empty:
        report = datasets
    else:
        report = pd.concat( [report, datasets] )

In [ ]:
from datetime import datetime
now = datetime.now() 

report = report[report['is_protected']==True] 
directory = 'uuid-protected-data-report'

if not Path(directory).exists():
    Path(directory).mkdir()
report.to_csv( directory + '/' + str(now.strftime('%Y%m%d')) + '.tsv', sep='\t', index=False )